In [1]:
# MET trigger efficiency measurement

import ROOT as rt
# import root_numpy as rtnp
import csv
import re
import sys
import collections
import os

from collections import OrderedDict
import uproot
import pandas as pd

import scipy
import awkward
import numpy as np
import time
import numba
from numba import jit
from matplotlib import pyplot as plt
sys.path.append('/storage/user/christiw/gpu/christiw/llp/delayed_jet_analyzer/lib/')
sys.path.append('/storage/user/christiw/login-1/christiw/LLP/CMSSW_9_4_4/src/llp_analyzer/python/')
from helper import make_datacard, make_datacard_2sig, weight_calc
from histo_utilities import create_TH1D, create_TH2D, std_color_list, create_TGraph, make_ratio_plot,make_ratio_pEff

import CMS_lumi, tdrstyle
tdrstyle.setTDRStyle()
CMS_lumi.writeExtraText = 0

wH = 1
Z_MASS = 91.2


# donotdelete = []
print(sys.version)

Welcome to JupyROOT 6.18/02
3.6.8 (default, Aug  7 2019, 17:28:10) 
[GCC 4.8.5 20150623 (Red Hat 4.8.5-39)]


In [2]:
def keyToyear(key):
    for year in years:
        if year in key:
            return year

# Load ntuples

In [4]:
fpath =OrderedDict()
tree = OrderedDict()


mc_years = ['Summer16', 'Fall17', 'Fall18']
data_years = ['Data2016', 'Data2017', 'Data2018ABC']
year =2
if year ==0:tune = 'TuneCUETP8M1'
else: tune = 'TuneCP5'
lumi = [35.92 * 1000, 41.53 * 1000, 59.74 * 1000,]
ntupler_version = 'V1p17/'
analyzer_version = '/v64/'


data_path = '/mnt/hadoop/store/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/V1p17/'+data_years[year]+'_AOD/v5/'+analyzer_version+'/normalized/'
if year == 0: 
    fpath['data'] = data_path + 'Run2_displacedJetMuonNtupler_V1p17_Data2016_AOD_Run2016-07Aug17_goodLumi.root'
elif year == 1:
    fpath['data'] = data_path + 'Run2_displacedJetMuonNtupler_V1p17_Data2017_AOD_Run2017-17Nov2017_goodLumi.root'
else: 
    fpath['data'] = data_path + 'Run2_displacedJetMuonNtupler_V1p17_Data2018_AOD_17Sept2018_Run2018-17Sep2018_goodLumi.root'
mc_path = '/mnt/hadoop/store/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/V1p17/MC_'+mc_years[year]+'/v2/'+analyzer_version+'/normalized/'
fpath['mc'] = mc_path+'WJetsToLNu_HT-70ToInf_'+tune+'_13TeV-madgraphMLM-pythia8_1pb_weighted.root'

NEvents = {}
NEvents_genweight = {}
for k,v in fpath.items():
    print(v)
    root_dir = uproot.open(v) 
    if not root_dir: 
        print(k, "zombie")
        continue
    tree[k] = root_dir['MuonSystem']
    NEvents[k] = root_dir['NEvents'][1]
    w = tree[k]["weight"].array()


/mnt/hadoop/store/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/V1p17/Data2018ABC_AOD/v5//v64//normalized/Run2_displacedJetMuonNtupler_V1p17_Data2018_AOD_17Sept2018_Run2018-17Sep2018_goodLumi.root
/mnt/hadoop/store/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/V1p17/MC_Fall18/v2//v64//normalized/WJetsToLNu_HT-70ToInf_TuneCP5_13TeV-madgraphMLM-pythia8_1pb_weighted.root


In [ ]:
/mnt/hadoop/store/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/V1p17/Data2016_AOD/v5//v64//normalized/Run2_displacedJetMuonNtupler_V1p17_Data2016_AOD_Run2016-07Aug17_goodLumi.root
/mnt/hadoop/store/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/V1p17/MC_Summer16/v2//v64//normalized/WJetsToLNu_HT-70ToInf_TuneCUETP8M1_13TeV-madgraphMLM-pythia8_1pb_weighted.root


/mnt/hadoop/store/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/V1p17/Data2017_AOD/v5//v64//normalized/Run2_displacedJetMuonNtupler_V1p17_Data2017_AOD_Run2017-17Nov2017_goodLumi.root
/mnt/hadoop/store/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/V1p17/MC_Fall17/v2//v64//normalized/WJetsToLNu_HT-70ToInf_TuneCP5_13TeV-madgraphMLM-pythia8_1pb_weighted.root

/mnt/hadoop/store/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/V1p17/Data2018ABC_AOD/v5//v64//normalized/Run2_displacedJetMuonNtupler_V1p17_Data2018_AOD_17Sept2018_Run2018-17Sep2018_goodLumi.root
/mnt/hadoop/store/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/V1p17/MC_Fall18/v2//v64//normalized/WJetsToLNu_HT-70ToInf_TuneCP5_13TeV-madgraphMLM-pythia8_1pb_weighted.root

# nCsc with different hit vetoing

In [49]:
%%time
sel_ev = {}
HLT_PFMET120_PFMHT120_IDTight = {}
HLT_PFMETNoMu120_PFMHTNoMu120_IDTight = {}
HLT_PFMET140_PFMHT140_IDTight = {}
HLT_PFMET120_PFMHT120_IDTight_PFHT60 = {}
HLT_PFMETNoMu120_PFMHTNoMu120_IDTight_PFHT60 = {}
HLT_PFMETNoMu140_PFMHTNoMu140_IDTight = {}
met = {}
metNoMu = {}
METTrigger = {}
weight = {}
lepPt = {}
HLT_PFMETNoMu = {}
HLT_PFMET = {}
for k, T in tree.items():

########### SELECTION: EVENTS ############

    sel_muon = np.logical_and(T.array('lepPt')<100, np.abs(T.array('lepPdgId'))==13)
#     sel_ele = np.logical_and(T.array('lepPt')<300, np.abs(T.array('lepPdgId'))==11)
    sel_ele = np.abs(T.array('lepPdgId'))==11


#     sel_ev[k] = np.logical_and(T.array('HLT_IsoMu27'),sel_muon.sum()>0)
    sel_ev[k] = np.logical_and(T.array('HLT_IsoEle'),sel_ele.sum()>0)


    sel_ev[k] = np.logical_and(sel_ev[k], T.array('Flag2_all'))



    HLT_PFMET120_PFMHT120_IDTight[k] = T.array('HLT_PFMET120_PFMHT120_IDTight')[sel_ev[k]]
    HLT_PFMETNoMu120_PFMHTNoMu120_IDTight[k] = T.array('HLT_PFMETNoMu120_PFMHTNoMu120_IDTight')[sel_ev[k]]
    HLT_PFMET140_PFMHT140_IDTight[k] = T.array('HLT_PFMET140_PFMHT140_IDTight')[sel_ev[k]]
    HLT_PFMET120_PFMHT120_IDTight_PFHT60[k] = T.array('HLT_PFMET120_PFMHT120_IDTight_PFHT60')[sel_ev[k]]
    HLT_PFMETNoMu120_PFMHTNoMu120_IDTight_PFHT60[k] = T.array('HLT_PFMETNoMu120_PFMHTNoMu120_IDTight_PFHT60')[sel_ev[k]]
    HLT_PFMETNoMu140_PFMHTNoMu140_IDTight[k] = T.array('HLT_PFMETNoMu140_PFMHTNoMu140_IDTight')[sel_ev[k]]
    if year == 0: HLT_PFMETNoMu[k] = HLT_PFMETNoMu120_PFMHTNoMu120_IDTight[k]
    else: HLT_PFMETNoMu[k] = np.logical_or(np.logical_or(HLT_PFMETNoMu120_PFMHTNoMu120_IDTight[k], HLT_PFMETNoMu120_PFMHTNoMu120_IDTight_PFHT60[k]), HLT_PFMETNoMu140_PFMHTNoMu140_IDTight[k])
    if year == 0: HLT_PFMET[k] = HLT_PFMET120_PFMHT120_IDTight[k]
    else: HLT_PFMET[k] = np.logical_or(np.logical_or(HLT_PFMET120_PFMHT120_IDTight[k], HLT_PFMET120_PFMHT120_IDTight_PFHT60[k]), HLT_PFMET140_PFMHT140_IDTight[k])

#     else:
#     sel_ev[k] = T.array('HLTDecision')[:,136]
#     HLT_PFMET120_PFMHT120_IDTight[k] = T.array('HLTDecision')[:,310][sel_ev[k]]
#     HLT_PFMETNoMu120_PFMHTNoMu120_IDTight[k] = T.array('HLTDecision')[:,467][sel_ev[k]]
#     HLT_PFMET140_PFMHT140_IDTight[k] = T.array('HLTDecision')[:,703][sel_ev[k]]
#     HLT_PFMET120_PFMHT120_IDTight_PFHT60[k] =T.array('HLTDecision')[:,709][sel_ev[k]]
#     HLT_PFMETNoMu120_PFMHTNoMu120_IDTight_PFHT60[k] = T.array('HLTDecision')[:,710][sel_ev[k]]
#     HLT_PFMETNoMu140_PFMHTNoMu140_IDTight[k] = T.array('HLTDecision')[:,717][sel_ev[k]]


#     lepPt[k] = T.array('lepPt')[sel_ev[k]][:,0]
    weight[k] = (T.array('weight')*T.array('pileupWeight'))[sel_ev[k]]
    if k == 'data':weight[k] = weight[k]*0.0+1.0
    METTrigger[k] = T.array('METTrigger')[sel_ev[k]]
    met[k] = T.array('met')[sel_ev[k]]
    metNoMu[k] = T.array('metNoMu')[sel_ev[k]]
    print(k,np.sum(weight[k]))

data 72188584.0
mc 236.49712
CPU times: user 47 s, sys: 18.8 s, total: 1min 5s
Wall time: 2min 21s


In [ ]:
import importlib
importlib.reload(sys.modules['histo_utilities'])
from histo_utilities import create_TH1D, create_TH2D, std_color_list, create_TGraph, make_ratio_plot, make_ratio_pEff

var = [met, metNoMu]
title = ['MET [GeV]', 'MET_NoMu [GeV]']
names = ['met', 'metNoMu']
yaxis = ['Trigger Efficiency', 'Events', 'Events']
cond_var = [HLT_PFMET,HLT_PFMETNoMu, METTrigger]
cond_name=['met_trig', 'metNoMu_trig', 'all_trig']
for plot in range(3):
    for j in range(len(title)):
        for m, cond in enumerate(cond_var):
#             if not(plot==0 and j == 1 and m == 2):continue
            c = rt.TCanvas('c','c', 800, 600)
            if plot ==0:leg = rt.TLegend(0.2,0.80,0.4,0.90)
            else: leg = rt.TLegend(0.7,0.80,0.9,0.90)
            leg.SetTextSize(0.03)
            leg.SetBorderSize(0)
            leg.SetEntrySeparation(0.01)
            hm = {}
            hb = {}
            pEff={}
            for i,k in enumerate(tree.keys()):
                if k == 'data': luminosity = 1
                else: luminosity = lumi[year]
                bins = list(np.arange(0,400,20))+[400,450, 600,700,800,1000]
#                 bins = [30,0,600]
#                 bins = [25,0,500]
#                 arr[arr > 255] = x
                temp = var[j][k]
#                 temp[temp>499] = 499
                hm[k] = create_TH1D(temp[cond[k]], 'hm1', axis_title = [title[j],yaxis[plot]], binning=bins, weights = weight[k][cond[k]]*luminosity) 
                hb[k] = create_TH1D(temp, 'hb1', axis_title = [title[j],yaxis[plot]], binning=bins, weights =  weight[k]*luminosity) 
#                 hm[k] = create_TH1D(np.abs(var[j][k][cond[k]]), 'hm1', axis_title = [title[j],yaxis[plot]], binning=bins, weights = weight[k][cond[k]]*luminosity) 
#                 hb[k] = create_TH1D(np.abs(var[j][k]), 'hb1', axis_title = [title[j],yaxis[plot]], binning=bins, weights =  weight[k]*luminosity) 
                pEff[k] = rt.TEfficiency(hm[k],hb[k])
                pEff[k].SetLineColor(std_color_list[i])
                hm[k].SetLineColor(std_color_list[i])
                hb[k].SetLineColor(std_color_list[i])
                pEff[k].SetLineWidth(2)
                hm[k].SetLineWidth(2)
                hb[k].SetLineWidth(2)
                print(k, luminosity, hb[k].Integral(),hm[k].Integral())
                if plot == 0:
                    leg.AddEntry(pEff[k],k)
                    pEff[k].Draw('' if i==0 else 'same')
                    file_name = 'efficiency'
                elif plot == 1:
                    leg.AddEntry(hb[k],k)
                    hb[k].Draw('hist same')
                    file_name = 'denominator'
                else:
                    leg.AddEntry(hm[k],k)
                    hm[k].Draw('hist same')
                    file_name = 'nominator'
            if plot == 0:
                c = make_ratio_pEff([pEff['mc'], pEff['data']], fit = False, logy=False, in_tags = ["WJets","Data"], ratio_bounds = [0,1.5], draw_opt = ['E1','E1'])

            elif plot == 1:
                c = make_ratio_plot([hb['mc'], hb['data']], fit = False, logy=True, in_tags = ["WJets","Data"], ratio_bounds = [0,3], draw_opt = ['E1','E1'])
            else:
                c = make_ratio_plot([hm['mc'], hm['data']], fit = False, logy=True, in_tags = ["WJets","Data"], ratio_bounds = [0,3], draw_opt = ['E1','E1'])
 

    #         leg.Draw()
            if plot>0:c.SetLogy()
            c.Draw()
            c.SaveAs('/storage/user/christiw/gpu/christiw/llp/delayed_jet_analyzer/plots/MuonSystem_Analysis/trigger_efficiency/'+analyzer_version+mc_years[year]+'_'+names[j]+"_"+cond_name[m]+'_'+file_name+'.png')
            c.SaveAs('/storage/user/christiw/gpu/christiw/llp/delayed_jet_analyzer/plots/MuonSystem_Analysis/trigger_efficiency/'+analyzer_version+mc_years[year]+'_'+names[j]+"_"+cond_name[m]+'_'+file_name+'.C')

# save ratio to ROOT file

In [50]:
outputDir = '/storage/user/christiw/login-1/christiw/LLP/CMSSW_9_4_4/src/llp_analyzer/data/ScaleFactors/'
outputFileName = outputDir+'METTriggers_SF.root'
# outFile = rt.TFile(outputFileName, 'UPDATE')





hm = {}
hb = {}
pEff={}
bins = [30,0,600]
bins = list(np.arange(0,400,20))+[400,450, 600,700,800,1000]


for i,k in enumerate(tree.keys()):
    cond = HLT_PFMETNoMu[k]
    if k == 'data': luminosity = 1
    else: luminosity = lumi[year]
    hm[k] = create_TH1D(np.abs(metNoMu[k][cond]), 'hm1', axis_title = ['MET','Efficiency'], binning=bins, weights = weight[k][cond]*luminosity) 
    hb[k] = create_TH1D(np.abs(metNoMu[k]), 'hb1', axis_title = ['MET','Efficiency'], binning=bins, weights =  weight[k]*luminosity) 
    pEff[k] = rt.TEfficiency(hm[k],hb[k])

    
    
    

nom = pEff['data'].GetCopyPassedHisto()
denom = pEff['mc'].GetCopyPassedHisto()
nom.SetLineColor(pEff['data'].GetLineColor())
#set relative error of ratio to be the relative error of data
for j in range(nom.GetXaxis().GetNbins()+1):
    nom.SetBinContent(j+1,pEff['data'].GetEfficiency(j+1))
    denom.SetBinContent(j+1,pEff['mc'].GetEfficiency(j+1))
    nom.SetBinError(j+1,max(pEff['data'].GetEfficiencyErrorLow(j+1), pEff['data'].GetEfficiencyErrorUp(j+1)))
    denom.SetBinError(j+1,max(pEff['mc'].GetEfficiencyErrorLow(j+1), pEff['mc'].GetEfficiencyErrorUp(j+1)))
nom.Divide(denom)
    
for i in range(nom.GetXaxis().GetNbins()+1):
    print(nom.GetBinCenter(i+1), nom.GetBinContent(i+1))

# Normalize(histPt)

# outFile.WriteTObject(nom, 'trigger_efficiency_'+mc_years[year], "WriteDelete");
# outFile.Close();

10.0 0.5235371597458819
30.0 0.573686985200462
50.0 0.3598441215951867
70.0 0.3648966851824489
90.0 0.4238907955560277
110.0 0.47703373601450894
130.0 0.5151279796514888
150.0 0.5666473016289832
170.0 0.6719704011340446
190.0 0.7740997268000165
210.0 0.8583412693047137
230.0 0.9114181753951717
250.0 0.9434488625816531
270.0 0.9570039450279302
290.0 0.9606727021788551
310.0 0.9642929525494447
330.0 0.9662979678457121
350.0 0.9618796861203044
370.0 0.9566440277319069
390.0 0.9608722967485552
425.0 0.9533138670585112
525.0 0.9580092314832613
650.0 0.9478094342730914
750.0 0.930077599087426
900.0 0.897688435079022
1020.0 0.7452043591476072


Warning in <TROOT::Append>: Replacing existing TH1: hm1 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: hb1 (Potential memory leak).
Info in <TEfficiency::TEfficiency>: given histograms are filled with weights
Info in <TROOT::TEfficiency::SetUseWeightedEvents>: Handle weighted events for computing efficiency
Warning in <TEfficiency::GetEfficiencyErrorLow>: frequentist confidence intervals for weights are only supported by the normal approximation
Info in <TEfficiency::GetEfficiencyErrorLow>: setting statistic option to kFNormal
